Data Ingestion 

In [1]:
import os
os.chdir("../")

%pwd

'c:\\Projects\\DSML\\ReactorExplorer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [ ]:
import sys
from reactorexplorer.constants import *
from reactorexplorer.utils.utils import read_yaml, create_directories
from reactorexplorer.exception.exception_handler import AppException

class ConfigurationManager:
    def __init__ (
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            schema_file_path = SCHEMA_FILE_PATH):
        try:
            self.config = read_yaml(CONFIG_FILE_PATH)
            self.paramls = read_yaml(PARAMS_FILE_PATH)
            self.schema = read_yaml(SCHEMA_FILE_PATH)

            create_directories([self.config.artifacts_root])

        except Exception as e:
            raise AppException(e, sys) from e 
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        try:
            config = self.config.data_ingestion
            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_url = config.source_url,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
            )

            return data_ingestion_config
        
        except Exception as e:
            raise AppException(e, sys) from e 

In [14]:
import os
import sys 
import urllib.request as request
import zipfile
from reactorexplorer.logger.log import logging
from reactorexplorer.exception.exception_handler import AppException
from reactorexplorer.utils.utils import get_size

class DataIngestion:
    def __init__(self, config = DataIngestionConfig):
        try:
            self.config = config
        except Exception as e:
            raise AppException(e, sys) from e
    
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url = self.config.source_url,
                    filename = self.config.local_data_file
                )

                logging.info(f"file downloaded: {self.config.local_data_file}")
            else:
                logging.info(f"file already exists")

        except Exception as e:
            raise AppException(e, sys) from e 

    def extract_zip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
          
        except Exception as e:
            raise AppException(e, sys) from e 

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file
except Exception as e:
    raise AppException(e, sys) from e 

AppException: Error occurred in script: [C:\Users\varsh\AppData\Local\Temp\ipykernel_34948\1377547025.py] at line number: [2] error message: [Error occurred in script: [C:\Users\varsh\AppData\Local\Temp\ipykernel_34948\2535024050.py] at line number: [17] error message: [Argument path_to_directories of type <class 'str'> to <function create_directories at 0x000002A87E8B4F40> does not match annotation type <class 'list'>]]